## Imports

In [51]:
import pandas as pd
import numpy as np

## Constants

In [99]:
VOCAB_SIZE = 2500

TRAINING_DATA_FILE = "SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE = "SpamData/03_Testing/prob-ham.txt"
TOKEN_ALL_PROB_FILE = "SpamData/03_Testing/prob-all-tokens.txt"

TEST_FEATURE_MATRIX = "SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE = "SpamData/03_Testing/test-target.txt"

## Read and Load Features

In [53]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter= " ", dtype= int)

In [54]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter= " ", dtype = int)

In [55]:
print (" NR of rows in training file: ", sparse_train_data.shape[0])
print (" NR of rows in test file: ", sparse_test_data.shape[0])
print ("NR of emails in training file: ", np.unique(sparse_train_data[:,0]).size)
print ("NR of emails in test file: ", np.unique(sparse_test_data[:,0]).size)

 NR of rows in training file:  239008
 NR of rows in test file:  109360
NR of emails in training file:  4011
NR of emails in test file:  1724


## Create an Empty DataFrame

In [56]:
column_names = ["DOC_ID"]+["CATEGORY"]+ list(range(0,VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [57]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [58]:
full_train_data = pd.DataFrame(index= index_names, columns = column_names)
full_train_data.fillna(value = 0)

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a Full Matrix From Sparse Matrix

In [59]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx = 0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas Dataframe.
    sparse matrix - np array
    nr_words - size of vocab
    doc_idx - psdition of doc
    cat_idx - position pf label
    freq-idx - position of occurance
    """

    column_names = ["DOC_ID"]+["CATEGORY"]+ list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index= doc_id_names, columns = column_names)
    full_matrix.fillna(value = 0, inplace = True)

    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr, "DOC_ID"] = doc_nr
        full_matrix.at[doc_nr, "CATEGORY"] = label
        full_matrix.at[doc_nr, word_id] = occurence

    full_matrix.set_index("DOC_ID", inplace = True)

    return full_matrix


In [60]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

In [61]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,2,0,1,3,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,7,2,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,1,3,2,14,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,5,2,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Bayes Model

#### Calculating Probability of Spam

In [62]:
full_train_data.CATEGORY.size

4011

In [63]:
full_train_data.CATEGORY.sum()

1247

In [64]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print("Probability of spam: ", round(prob_spam,3))

Probability of spam:  0.311


#### Total Number of Words (Tokens)

In [65]:
full_train_features = full_train_data.loc[:, full_train_data.columns != "CATEGORY"]
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,1,2,0,1,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,7,2,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,6,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,1,3,2,14,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5791,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
email_lengths = full_train_features.sum(axis = 1)
email_lengths.shape

(4011,)

In [67]:
total_wc = email_lengths.sum()
total_wc

375661

## Number f Tokens in Spam and Ham Email

In [68]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1247,)

In [69]:
spam_wc = spam_lengths.sum()
spam_wc

157744

In [70]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2764,)

In [71]:
ham_wc = ham_lengths.sum()
ham_wc

217917

In [76]:
print("Average number of words in spams: ",round (spam_wc/spam_lengths.shape[0],))
print("Average number of words in hams: ",round (ham_wc/ham_lengths.shape[0],))

Average number of words in spams:  126
Average number of words in hams:  79


## Summing the Tokens Occuring in Spam

In [77]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]

In [78]:
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,1,2,0,1,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,7,2,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,6,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,1,3,2,14,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens

0       2042
1       1007
2       1832
3        951
4       1089
        ... 
2495       4
2496      29
2497      18
2498       5
2499      25
Length: 2500, dtype: int64

In [85]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis = 0 ) + 1
summed_ham_tokens

0       4801
1       1893
2        894
3       1619
4       1272
        ... 
2495      37
2496      11
2497       7
2498      38
2499       4
Length: 2500, dtype: int64

## P(Token | Spam) - Token in given spam Probability

In [89]:
prob_tokens_spam = summed_spam_tokens/(spam_wc + VOCAB_SIZE)
prob_tokens_spam.sum()

1.0

## P(Token | HAM) - Token in given non-spam Probability

In [90]:
prob_tokens_ham = summed_ham_tokens/(ham_wc + VOCAB_SIZE)
prob_tokens_ham.sum()

1.0

## P(Token) Probability that Token Occurs

In [92]:
prob_tokens_all = full_train_features.sum(axis = 0) / total_wc
prob_tokens_all.sum()

1.0

## Save Trained Model

In [98]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [100]:
full_test_data = make_full_matrix(sparse_test_data, nr_words= VOCAB_SIZE)

In [103]:
x_test = full_test_data.loc[:, full_test_data.columns != "CATEGORY"]
y_test = full_test_data.CATEGORY

In [104]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, x_test)